In [1]:
#include <iostream>
#include <thread>
#include <future>

# 1. Promises and Futures

## The promise - future communication channel

In [ ]:
void modifyMessage(std::promise<std::string> && prms, std::string message)
{
    std::this_thread::sleep_for(std::chrono::milliseconds(4000)); // simulate work
    std::string modifiedMessage = message + " has been modified"; 
    prms.set_value(modifiedMessage);
}

In [ ]:
std::string messageToThread = "My Message";

// create promise and future
std::promise<std::string> prms;
std::future<std::string> ftr = prms.get_future();

// start thread and pass promise as argument
std::thread t(modifyMessage, std::move(prms), messageToThread);

// print original message to console
std::cout << "Original message from main(): " << messageToThread << std::endl;

// retrieve modified message via future and print to console
std::string messageFromThread = ftr.get();
std::cout << "Modified message from thread(): " << messageFromThread << std::endl;

In [2]:
void divideByNumber(std::promise<double> &&prms, double num, double denom)
{
    std::this_thread::sleep_for(std::chrono::milliseconds(500)); // simulate work
    try
    {
        if (denom == 0)
            throw std::runtime_error("Exception from thread: Division by zero!");
        else
            prms.set_value(num / denom);
    }
    catch (...)
    {
        prms.set_exception(std::current_exception());
    }
}

In [3]:
std::promise<double> prms;
std::future<double> ftr = prms.get_future();

// start thread and pass promise as argument
double num = 42.0, denom = 0.0;
std::thread t(divideByNumber, std::move(prms), num, denom);

// retrieve result within try-catch-block
try
{
    double result = ftr.get();
    std::cout << "Result = " << result << std::endl;
}
catch (std::runtime_error e)
{
    std::cout << e.what() << std::endl;
}

IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt11__once_call' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt15__once_callable' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!


Standard Exception: std::future_error: No associated state

# 2. Threads vs. Tasks

# Starting threads with async


The first change we are making is in the thread function: We are removing the promise from the argument list as well as the try-catch block. Also, the return type of the function is changed from void to double as the result of the computation will be channeled back to the main thread using a simple return. After these changes, the function has no knowledge of threads, nor of futures or promises - it is a simple function that takes two doubles as arguments and returns a double as a result. Also, it will throw an exception when a division by zero is attempted.

one of the major differences between std::thread and std::async is that with the latter, the system decides wether the associated function should be run asynchronously or synchronously. By adjusting the launch parameters of std::async manually, we can directly influence wether the associated thread function will be executed synchronously or asynchronously.

To conclude this section, a general comparison of task-based and thread-based programming is given in the following:

With tasks, the system takes care of many details (e.g. join). With threads, the programmer is responsible for many details. As far as resources go, threads are usually more heavy-weight as they are generated by the operating system (OS). It takes time for the OS to be called and to allocate memory / stack / kernel data structures for the thread. Also, destroying the thread is expensive. Tasks on the other hand are more light-weight as they will be using a pool of already created threads (the "thread pool").

Threads and tasks are used for different problems. Threads have more to do with latency. When you have functions that can block (e.g. file input, server connection), threads can avoid the program to be blocked, when e.g. the server is waiting for a response. Tasks on the other hand focus on throughput, where many operations are executed in parallel.

The function template async runs the function f asynchronously (potentially in a separate thread which may be part of a thread pool) and returns a std::future that will eventually hold the result of that function call.



In [ ]:
double divideByNumberAsync(double num, double denom)
{
    std::this_thread::sleep_for(std::chrono::milliseconds(500)); // simulate work

    if (denom == 0)
        throw std::runtime_error("Exception from thread: Division by zero!");

    return num / denom;
}


## Policies
- the async flag is set "asynchronous execution"
- the deferred flag is set "lazy evaluation"
- If both the std::launch::async and std::launch::deferred flags are set in policy, it is up to the implementation whether to perform asynchronous execution or lazy evaluation.
- If neither std::launch::async nor std::launch::deferred, nor any implementation-defined policy flag is set in policy, the behavior is undefined.

In [ ]:
double num = 42.0, denom = 2.0;
std::future<double> ftr = std::async(std::launch::deferred, divideByNumber, num, denom);

// retrieve result within try-catch-block
try
{
    double result = ftr.get();
    std::cout << "Result = " << result << std::endl;
}
catch (std::runtime_error e)
{
    std::cout << e.what() << std::endl;
}